In [1]:
# load packages
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [15]:
np.sin(90*np.pi/180)

1.0

In [18]:
def sindeg(deg):
    return np.sin(deg*np.pi/180)

In [19]:
sindeg(45)

0.7071067811865476

In [48]:
#Loaded weight need to be assumed
Route_length = 30 #miles
Mile_to_feet = 5280
Train_Mass = 3500 #Empty Train
Rolling_resistance = 5
ton_to_lbs = 2000
Slope = 0.005
Battery_capacity = 4.8 * 10 ** 6
Battery_efficiency = 0.8
kg_to_lbs = 2.204
MWh_to_footlb = 2655.22*10 ** 6
Speed = 30 #mph

In [101]:
def X_n_Z_n_updater(t,V,G):
    #t in second, v in mph, G in percentage
    V = V/3600*5280 # in feet/sec
    x = V * t
    z = V * t * G
    return x, z

def energy_needed(W, R, dX, dZ, a=1, b=ton_to_lbs):
    
    W = Train_Mass
    R = Rolling_resistance
    a = 1
    b = ton_to_lbs
    E_n = a * dX * R * W + b * W * dZ
    #Units: E_n: in distance*force format foot-pound
    #dX = feet
    #R = lbs/ton
    #W = tons
    # b: 
    # dZ: feet
    
    #OR from textbook's tractive resistance formula, from a power(V,G,W)*D perspective
    
    return E_n

def SOC_updater(Num_Loco, SOC, E_n, surplus,efficiency=Battery_capacity,capacity=Battery_efficiency):
    current = Num_Loco * capacity * SOC * MWh_to_footlb #transform to foot-pound
    if E_n > 0:
        update = current - E_n
        SOC = update/capacity/MWh_to_footlb/Num_Loco
    else:
        update = current - E_n * efficiency
        SOC = update/capacity/MWh_to_footlb/Num_Loco
        if SOC >= 1:
            surplus += (SOC-1) * capacity * MWh_to_footlb
            SOC = 1
    return SOC, update, surplus

In [114]:
#Initialization

X_n = X_0 = 0 #feet
Z_n = Z_0 = 0 #feet
SOC = 1
Slope = 0.005
Num_BEL = 4
surplus = 0 # foot-pound
t_total = int(Route_length/Speed*3600) #second
Output = pd.DataFrame(
        np.zeros(shape=(t_total+1, 4)), columns=["Time(sec)","Distance(feet)","SOC","Surplus(Foot-pound)"])

#Simulation Sequence
for t in range(t_total):
    X_t, Z_t = X_n_Z_n_updater(t,Speed,Slope)
    dX = X_t - X_n
    dZ = Z_t - Z_n
    Z_n = Z_t
    X_n = X_t
    E_n = energy_needed(Train_Mass, Rolling_resistance, dX, dZ)
    SOC, update, surplus = SOC_updater(Num_BEL,SOC,E_n,surplus)
    if SOC <= 0:
        SOC = 0
    Output.loc[t,"Time(sec)"] = t
    Output.loc[t,"Distance(feet)"] = X_n
    Output.loc[t,"SOC"] = SOC
    Output.loc[t,"Surplus(Foot-pound)"] = surplus
    

In [115]:
Output

,Time(sec),Distance(feet),SOC,Surplus(Foot-pound)
0,0.0,0.0,1.000000,0.0
1,1.0,44.0,0.999728,0.0
2,2.0,88.0,0.999456,0.0
3,3.0,132.0,0.999184,0.0
4,4.0,176.0,0.998913,0.0
...,...,...,...,...
3596,3596.0,158224.0,0.022355,0.0
3597,3597.0,158268.0,0.022083,0.0
3598,3598.0,158312.0,0.021811,0.0
3599,3599.0,158356.0,0.021539,0.0


In [120]:
#Initialization

X_n = X_0 = 0 #feet
Z_n = Z_0 = 0 #feet
SOC = 0.6
Num_BEL = 3
Slope = -0.00251
surplus = 0 # foot-pound
t_total = int(Route_length/Speed*3600) #second
Output = pd.DataFrame(
        np.zeros(shape=(t_total+1, 4)), columns=["Time(sec)","Distance(feet)","SOC","Surplus(Foot-pound)"])

#Simulation Sequence
for t in range(t_total):
    X_t, Z_t = X_n_Z_n_updater(t,Speed,Slope)
    dX = X_t - X_n
    dZ = Z_t - Z_n
    Z_n = Z_t
    X_n = X_t
    E_n = energy_needed(Train_Mass, Rolling_resistance, dX, dZ)
    SOC, update, surplus = SOC_updater(Num_BEL,SOC,E_n,surplus)
    Output.loc[t,"Time(sec)"] = t
    Output.loc[t,"Distance(feet)"] = X_n
    Output.loc[t,"SOC"] = SOC
    Output.loc[t,"Surplus(Foot-pound)"] = surplus
    

In [121]:
Output

,Time(sec),Distance(feet),SOC,Surplus(Foot-pound)
0,0.0,0.0,0.6,0.000000e+00
1,1.0,44.0,1.0,4.078330e+09
2,2.0,88.0,1.0,9.006330e+09
3,3.0,132.0,1.0,1.393433e+10
4,4.0,176.0,1.0,1.886233e+10
...,...,...,...,...
3596,3596.0,158224.0,1.0,1.772024e+13
3597,3597.0,158268.0,1.0,1.772517e+13
3598,3598.0,158312.0,1.0,1.773009e+13
3599,3599.0,158356.0,1.0,1.773502e+13
